In [ ]:
import re
import sqlite3

gazetteer = {
    'Lancaster, Pennsylvania': {'latitude': 40.0379, 'longitude': -76.3055},
    'Lancaster, England': {'latitude': 54.0500, 'longitude': -2.8000},
    'Philadelphia, Pennsylvania': {'latitude': 39.9526, 'longitude': -75.1652},
    'Harrisburg, Pennsylvania': {'latitude': 40.2737, 'longitude': -76.8844}
}

def extract_toponyms(text):
    toponyms = re.findall(r'\b[A-Z][a-z]*\s[A-Z][a-z]*\b', text)
    return toponyms

def resolve_toponym(toponym):
    if toponym in gazetteer:
        return gazetteer[toponym]
    return None

conn = sqlite3.connect('historical_geodata.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS Locations
                  (Toponym TEXT, Latitude REAL, Longitude REAL, Context TEXT)''')

def store_location(toponym, latitude, longitude, context):
    cursor.execute('INSERT INTO Locations (Toponym, Latitude, Longitude, Context) VALUES (?, ?, ?, ?)',
                   (toponym, latitude, longitude, context))
    conn.commit()

text = "The city of Lancaster is located in Pennsylvania, near Harrisburg and Philadelphia."

toponyms = extract_toponyms(text)
print("Extracted Toponyms:", toponyms)

for toponym in toponyms:
    print(f"Resolving toponym: {toponym}")

    resolved_location = resolve_toponym(toponym)

    if resolved_location:
        latitude = resolved_location['latitude']
        longitude = resolved_location['longitude']
        print(f"Resolved Location: Latitude = {latitude}, Longitude = {longitude}")

        store_location(toponym, latitude, longitude, text)
    else:
        print(f"Toponym: {toponym} could not be resolved.")

cursor.execute('SELECT * FROM Locations')
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

Extracted Toponyms: []
